# Introduction

This note implement the concept in https://towardsdatascience.com/gguf-quantization-with-imatrix-and-k-quantization-to-run-llms-on-your-cpu-02356b531926

This note will quantize Meta-Llama with Imatrix and K-quantization will llama.cpp

# Install llama.cpp

we use llama-cpp-python binding instead of the directly using llama-cpp


In [1]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
# !export GGML_CUDA=1 
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 
!make GGML_CUDA=1 CUDA_PATH=/usr/local/nvidia  > make.log 2>&1


Cloning into 'llama.cpp'...
remote: Enumerating objects: 34659, done.
remote: Counting objects: 100% (6648/6648), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 34659 (delta 6513), reused 6405 (delta 6383), pack-reused 28011 (from 1)
Receiving objects: 100% (34659/34659), 57.99 MiB | 14.30 MiB/s, done.
Resolving deltas: 100% (25153/25153), done.
/kaggle/working/llama.cpp


In [2]:
!tail make.log

c++ -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -DGGML_CUDA_USE_GRAPHS -I/usr/local/nvidia/include -I/usr/local/nvidia/targets/x86_64-linux/include  -c examples/cvector-generator/cvector-generator.cpp -o examples/cvector-generator/cvector-generator.o
c++ -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -DGGML_CUDA_USE_GRAPHS -I/usr/local/nvidi

In [3]:
!pwd
!pip install --force-reinstall -r requirements.txt > pip_install.log 2>&1
!tail pip_install.log

/kaggle/working/llama.cpp
pointpats 2.5.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
rapids-dask-dependency 24.8.0a0 requires dask==2024.7.1, but you have dask 2024.8.1 which is incompatible.
rmm 24.8.2 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
s3fs 2024.6.1 requires fsspec==2024.6.1.*, but you have fsspec 2024.9.0 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflow-metadata 0.14.0 requires protobuf<4,>=3.7, but you have protobuf 4.25.5 which is incompatible.
tensorflow-transform 0.14.0 requires protobuf<4,>=3.7, but you have protobuf 4.25.5 which is incompatible.
ydata-profiling 4.9.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.0 which is incompatible.


# Download model



In [4]:
!pwd
%cd ..
!mkdir "./quantized_model_gemma2-2b/"

/kaggle/working/llama.cpp
/kaggle/working


In [5]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from huggingface_hub import snapshot_download

user_secrets = UserSecretsClient()

os.environ["HF_TOKEN"]=user_secrets.get_secret("HUGGINGFACE_TOKEN")

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["MODEL_NAME"] = "google/gemma-2-2b-it"
os.environ["DATASET"] = "HuggingFaceH4/ultrafeedback_binarized"

login(os.environ["HF_TOKEN"])

model_name = "google/gemma-2-2b-it" # the model we want to quantize
methods = ['Q4_K_S','Q4_K_M'] #the methods to be used for quantization
base_model = "./original_model_gemma2-2b/" # where the FP16 GGUF model will be stored
quantized_path = "./quantized_model_gemma2-2b/" #where the quantized GGUF model will be stored
original_model = quantized_path + 'FP16.gguf'

snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1204: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

'/kaggle/working/original_model_gemma2-2b'

## Convert model to gguf

In [6]:
!python llama.cpp/convert_hf_to_gguf.py "./original_model_gemma2-2b/" --outfile "./quantized_model_gemma2-2b/FP16.gguf"

Writing: 100%|███████████████████████████| 5.23G/5.23G [00:22<00:00, 234Mbyte/s]


## Download the datasets that will be used for calibration and evaluation:

In [7]:
!wget https://object.pouta.csc.fi/OPUS-Wikipedia/v1.0/mono/en.txt.gz
!gunzip en.txt.gz
!head -n 10000 en.txt > en-h10000.txt
!sh llama.cpp/scripts/get-wikitext-2.sh

--2024-09-25 08:12:17--  https://object.pouta.csc.fi/OPUS-Wikipedia/v1.0/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 532958396 (508M) [application/gzip]
Saving to: 'en.txt.gz'

en.txt.gz           100%[===================>] 508.27M  9.25MB/s    in 57s     

2024-09-25 08:13:16 (8.86 MB/s) - 'en.txt.gz' saved [532958396/532958396]

--2024-09-25 08:13:32--  https://huggingface.co/datasets/ggml-org/ci/resolve/main/wikitext-2-raw-v1.zip
Resolving huggingface.co (huggingface.co)... 108.158.20.65, 108.158.20.70, 108.158.20.31, ...
Connecting to huggingface.co (huggingface.co)|108.158.20.65|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/c6/78/c67802fcd48fa6f6a86773410b21cc6db1c5c546b20683b6c30b95f327a66922/ef7edb566e3e2b2d31b29

# Compute the importance matrix

In [8]:
!./llama.cpp/llama-imatrix -m "./quantized_model_gemma2-2b/FP16.gguf"  -f en-h10000.txt -o quantized_model_gemma2-2b/imatrix.dat --verbosity 1 -ngl 99

build: 3823 (3d6bf691) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
llama_model_loader: loaded meta data with 38 key-value pairs and 288 tensors from ./quantized_model_gemma2-2b/FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model_Gemma2 2b
llama_model_loader: - kv   3:                           general.basename str              = original_model_gemma2
llama_model_loader: - kv   4:                         general.size_label str              = 2B
llama_model_loader: - kv   5:                            general.license str              = gemma
llama_model_loader: - kv   6:               

# Quantize the model with K-Quants

In [9]:
!./llama.cpp/llama-quantize --imatrix ./quantized_model_gemma2-2b/imatrix.dat "./quantized_model_gemma2-2b/FP16.gguf"  "./quantized_model_gemma2-2b/FP16_I.gguf" "Q4_K_S"

load_imatrix: imatrix dataset='en-h10000.txt'
load_imatrix: loaded 182 importance matrix entries from ./quantized_model_gemma2-2b/imatrix.dat computed on 696 chunks
prepare_imatrix: have 182 importance matrix entries
main: build = 3823 (3d6bf691)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing './quantized_model_gemma2-2b/FP16.gguf' to './quantized_model_gemma2-2b/FP16_I.gguf' as Q4_K_S
llama_model_loader: loaded meta data with 38 key-value pairs and 288 tensors from ./quantized_model_gemma2-2b/FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model_Gemma2 2b
lla

## Save result model

In [14]:
!pwd
!ls -lh quantized_model_gemma2-2b/
!gzip quantized_model_gemma2-2b/FP16_I.gguf

/kaggle/working
total 6.5G
-rw-r--r-- 1 root root 4.9G Sep 25 08:12 FP16.gguf
-rw-r--r-- 1 root root 1.6G Sep 25 08:34 FP16_I.gguf
-rw-r--r-- 1 root root 2.3M Sep 25 08:30 imatrix.dat
